# Extract Object Files from Tempest RK05 Disk after Assembling and Linking

When we follow the steps at [../atari_build/README.md](../atari_build/README.md) we have a file `tempest_original.rk05` with both the sources and the assembled object files on it. What we want to do here is extract the object files from `tempest_original.rk05`.

### Radix 50 Helper Functions

In [1]:
import math

CHARS = " ABCDEFGHIJKLMNOPQRSTUVWXYZ$.*0123456789"

def radix50_decode(v):
    if v > 63999:
        raise Exception("Invalid Value")
    c1 = math.floor(v/(40**2))
    v2 = v%(40**2)
    c2 = math.floor(v2/40)
    c3 = v2%40
    return CHARS[c1]+CHARS[c2]+CHARS[c3]

def radix50_encode(chars):
    if len(chars) != 3:
        raise Exception("Must be 3 characters")
    result = ((CHARS.index(chars[0]) * (40**2)) + 
              (CHARS.index(chars[1]) * (40)) + 
              CHARS.index(chars[2]))
    return result

print(radix50_encode("ABC"),radix50_encode("ABC") == 1683)
print(radix50_encode("DEF"),radix50_encode("ABC") == 6606)

print(radix50_decode(1683),radix50_decode(1683) == "ABC")
print(radix50_decode(6606),radix50_decode(6606) == "DEF")


1683 True
6606 False
ABC True
DEF True


### Parse an rk05 cartridge image file and dump its contents
See `AA-PD6PA-TC_RT-11_Volume_and_File_Formats_Manual_Aug91.pdf` in `../materials`

If you've followed the steps at [../atari_build/README.md](../atari_build/README.md) you now have a `tempest_original.rk05` that contains the object files generated by MAC65 and LINKM. We can extract them using the steps below.

First we copy the file over to this directory:

In [8]:
!cp ../atari_build/tempest_original.rk05 tempest_with_object_files.rk05

Now we can extract all the files in it to our `rk05_tempest_object_files` directory.

In [9]:
EOS_MARKER = 0x0800
EMPTY = 0x0200
TENTATIVE = 0x0100

rk1 = open("tempest_with_object_files.rk05",'rb')
header = rk1.read(0xc00)
no_segments = int.from_bytes(rk1.read(2), "little")
segment_no = int.from_bytes(rk1.read(2), "little")
segment_highest = int.from_bytes(rk1.read(2), "little")
extra_bytes = int.from_bytes(rk1.read(2), "little")
start_block = int.from_bytes(rk1.read(2), "little") # blocks are 512 bytes

segment_header = [
    no_segments,
    segment_no,
    segment_highest,
    extra_bytes,
    start_block
]
entries = []
while True:
    status_word = int.from_bytes(rk1.read(2), "little")
    if not status_word or status_word == EOS_MARKER:
        break
        
    first_three_chars = radix50_decode(int.from_bytes(rk1.read(2), "little"))
    second_three_chars = radix50_decode(int.from_bytes(rk1.read(2), "little"))
    third_three_chars = radix50_decode(int.from_bytes(rk1.read(2), "little"))
    no_octal_blocks = int.from_bytes(rk1.read(2), "little")
    reserved = int.from_bytes(rk1.read(2), "little")
    file_date = int.from_bytes(rk1.read(2), "little")
    
    entries += [[
        status_word,
        first_three_chars,
        second_three_chars,
        third_three_chars,
        no_octal_blocks,
        reserved,
        file_date
    ]]
#print("segment header",segment_header)

rk1.seek(start_block * 512)
for entry in entries:
    #print("entry", entry)
    file_name = ''.join(entry[1:3])+'.'+entry[3]
    file_length = entry[4] * 512
    file_data = rk1.read(file_length)
    if entry[0] == EMPTY or entry[0] == TENTATIVE:
        file_name += "_UNUSED"
        
    #print(file_name,file_data[:10],hex(rk1.tell()))
    open(f"rk05_tempest_object_files/{file_name}",'wb').write(file_data)
    